# Dependencies

In [1]:
pacman::p_load(
    ggpubr,
    igraph, graphlayouts, ggraph, ggforce, dplyr, 
    ggplot2, GUniFrac, ggsignif,
    dendextend, ComplexHeatmap, ANCOMBC, 
    TreeSummarizedExperiment, lsmeans, factoextra
)


# Functions

In [2]:
source("/Users/anabbi/git/ped_CapTCRseq/R/ggplot2_theme.R")
source("/Users/anabbi/git/ped_CapTCRseq/R/color_schemes.R")
source("/Users/anabbi/git/ped_CapTCRseq/R/Misc_functions.R")

# Paths

In [3]:
datapath <- "/Users/anabbi/OneDrive - UHN/Documents/INTERCEPT/Data/"
plotpath <- "/Users/anabbi/OneDrive - UHN/Documents/INTERCEPT/Plots/"
manifestpath <- "/Users/anabbi/OneDrive - UHN/Documents/INTERCEPT/Manifests/"
gitpath <- "/Users/anabbi/git/ped_CapTCRseq/"

In [4]:
h4hpath <- "/Users/anabbi/Desktop/H4H/INTERCEPT/analysis/GLIPH/run_gliph/"

# Main

In [5]:
# based on v5 gliph run

In [6]:
meta_div <- readr::read_rds(paste0(datapath, "capTCRseq/meta_div_goodsamples.rds"))

In [7]:
#remove those samples that we did not have the TCRseq data for
meta_div <- meta_div[!is.na(meta_div$TRB),]

In [8]:
gliph_out <- readr::read_rds(file = paste0(datapath, "GLIPH/int_gliph_v5_cleaned.rds"))

In [9]:
samples_tcr <- readr::read_rds(file = paste0(datapath, "GLIPH/int_samples_gliphannot_stat.rds"))
gliphdf <- readr::read_rds(file = paste0(datapath, "GLIPH/gliphannot_stat.rds"))

In [10]:
head(gliphdf)

,gliph,CMV,EBV,HCV,Influenza,HomoSapiens,MDavis,HPV,CEF,TB,MCPyV,S-pneumoniae,n_unique_cdr3
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
SLG%E,SLG%E,0,0,0,0,0,39,0,0,0,0,0,50
S%GGNTE,S%GGNTE,0,0,0,0,1,40,0,0,0,0,0,41
GIWV,GIWV,0,0,0,0,0,0,0,0,0,0,0,82
IWV,IWV,0,0,0,0,1,0,0,0,0,0,0,97
WVYE,WVYE,0,0,0,0,0,0,0,0,0,0,0,98
IWVY,IWVY,0,0,0,0,0,0,0,0,0,0,0,95


In [11]:
head(samples_tcr)

,CMV,EBV,HCV,Influenza,HomoSapiens,MDavis,HPV,CEF,TB,MCPyV,...,cdr3_reads,n_gliph,n_cdr3ingliph,cdr3reads_ingliph,cancergroup,cycle,sampletype,cyclenum,Patient,Age
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,...,<int>,<dbl>,<int>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<chr>,<dbl>
Leukemia:CHP_319_X05-PBMC,0,0,0,0,0,2,0,0,0,0,...,79,3,3,9,Leukemia,X05,PBMC,5,CHP_319,1
Leukemia:CHP_323_X03-PBMC,2,0,1,0,0,16,0,0,0,1,...,3175,21,12,1537,Leukemia,X03,PBMC,3,CHP_323,1
Leukemia:CHP_323_X03-cfDNA,2,0,0,0,4,33,0,0,0,0,...,81588,49,25,51235,Leukemia,X03,cfDNA,3,CHP_323,1
Leukemia:CHP_323_X05-PBMC,1,0,0,0,2,47,0,0,0,1,...,2020,71,36,777,Leukemia,X05,PBMC,5,CHP_323,1
Leukemia:CHP_323_X05-cfDNA,0,0,0,0,0,4,0,0,0,0,...,4938,6,2,3273,Leukemia,X05,cfDNA,5,CHP_323,1
Leukemia:CHP_326_X05-cfDNA,0,0,3,0,0,5,0,0,0,0,...,18584,8,4,7553,Leukemia,X05,cfDNA,5,CHP_326,4


In [13]:
gliph_out_chp <- gliph_out[ grepl("CHP", gliph_out$Sample),]

In [10]:
components_sep_cfdna <- readr::read_rds(paste0(datapath, "GLIPH/components_sep_cfDNA.rds"))
components_sep_pbmc <- readr::read_rds(paste0(datapath, "GLIPH/components_sep_PBMC.rds"))

In [ ]:
pat_type <- gliph_out_nosingle_chppatterns[, c("pattern", "Sample", "Freq")] %>%
  tidyr::pivot_wider(names_from = Sample, values_from = Freq, values_fn = sum, values_fill = 0)
pat_type <- as.data.frame(pat_type)
rownames(pat_type) <- pat_type$pattern
pat_type$pattern <- NULL

In [ ]:
levels(as.factor(gliphdf$Family))

In [ ]:
# species = gliph, genus = original annotations, family = grouped annotations
gliphdf$Genus <- gliphdf$Family

In [ ]:
gliphdf$Family[grepl("hs_", gliphdf$Family)] <- "cross-reactive"
gliphdf$Family[grepl("_hs", gliphdf$Family)] <- "cross-reactive"
gliphdf$Family[ gliphdf$Family %in% c("CMV", "EBV", "HPV")] <- "Viral"

In [ ]:
levels(as.factor(gliphdf$Genus))
levels(as.factor(gliphdf$Family))

Create TreeSummarizedExperiment to input to ANCOMBC

In [ ]:
table(metadata_matched$cancergroup)

In [ ]:
rownames(metadata_matched) <- metadata_matched$samplename

In [ ]:
head(metadata_matched)

In [ ]:
table(gliphdf$Family)

In [ ]:
head(gliphdf)

In [ ]:
tse_X01 <- TreeSummarizedExperiment(
    assays = list(Count = as.matrix(pat_type)),
    rowData = gliphdf,
    colData = metadata_matched
)

In [ ]:
table(metadata_matched$cancergroup)

In [ ]:
# tse_X01 <- onetest_tse[, colnames(onetest_tse)[grepl("X01", colnames(onetest_tse)) ] ]

# tse_X01 <- tse_X01[, colnames(tse_X01)[!grepl("T-cellmalignancies", colnames(tse_X01)) ] ]

In [ ]:
nks <- gliphdf$gliph[ gliphdf$Family %in% c("NK", "TB")]

In [ ]:
length(nks)

In [ ]:
tse_X01 <- tse_X01[ !rownames(tse_X01) %in% nks, ]

In [ ]:
tse_X01

In [ ]:
table(rowData(tse_X01)$Family)

In [ ]:
# colData(tse_X01)$cancergroup <- as.character(colData(tse_X01)$cancergroup)

# colData(tse_X01) <- colData(tse_X01)[ colData(tse_X01)$cancergroup != "T-cell malignancies"]

In [ ]:
colData(tse_X01)$cancergroup <- factor(colData(tse_X01)$cancergroup, levels = c("Leukemia", "Lymphoma", "Solid tumors"))

In [ ]:
table(rowData(tse_X01)$gliph)

In [ ]:
# readr::write_rds(onetest_tse, "/Users/anabbi/Desktop/H4H/INTERCEPT/analysis/GLIPH/ancombc/int_gliph_tse_X01_pbmc.rds")

In [ ]:
# rowData(tse_X01)$Family <- rowData(tse_X01)$gliph

In [ ]:
set.seed(111)

In [ ]:
out <- ancombc2(
    data = tse_X01, assay_name = "Count",
    tax_level = NULL, 
    fix_formula = "cancergroup", rand_formula = NULL,#"(1|Patient)",
    p_adj_method = "fdr", 
    pseudo = 0, pseudo_sens = TRUE, # add pseudocounts
    prv_cut = 0.01, # prevalence cutoff
    lib_cut = 0, # library size cutoff
    s0_perc = 0.05, 
    group = "cancergroup", 
    struc_zero = FALSE, 
    neg_lb = FALSE, # check this again
    alpha = 0.05, 
    n_cl = 10, # number of nodes
    verbose = TRUE,
    global = TRUE, pairwise = TRUE, dunnet = TRUE, trend = FALSE, # test types
    iter_control = list(tol = 1e-2, max_iter = 20, verbose = TRUE),
    em_control = list(tol = 1e-5, max_iter = 100),
    lme_control = lme4::lmerControl(),
    mdfdr_control = list(fwer_ctrl_method = "fdr", B = 100)
)


In [ ]:
myres <- out$res_pair

In [ ]:
myres

In [ ]:
out$res